<a href="https://colab.research.google.com/github/sumanlaraee/AI-ML/blob/main/VAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch torchaudio
!pip install -q git+https://github.com/snakers4/silero-vad


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
video_path = "/content/zoom_sample.mp4"
!ffmpeg -y -i "{video_path}" -ar 16000 -ac 1 -vn -acodec pcm_s16le audio.wav


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import torch
import torchaudio

# Download Silero VAD utils
from torch.hub import load

# Load the Silero VAD model and utils
model, utils = load(
    repo_or_dir='snakers4/silero-vad',
    model='silero_vad',
    source='github',
    trust_repo=True
)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils


Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


In [ ]:
# Load the audio file
wav = read_audio("audio.wav", sampling_rate=16000)

# Get voice timestamps
speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=16000)

print(speech_timestamps)


[{'start': 188448, 'end': 214496}, {'start': 220704, 'end': 273888}, {'start': 285216, 'end': 304096}, {'start': 311328, 'end': 374240}, {'start': 376864, 'end': 395744}, {'start': 397344, 'end': 443872}, {'start': 445472, 'end': 500704}, {'start': 504864, 'end': 528352}, {'start': 560672, 'end': 685536}, {'start': 909856, 'end': 921056}, {'start': 982560, 'end': 995296}, {'start': 1031712, 'end': 1099744}, {'start': 1104928, 'end': 1138656}, {'start': 1143328, 'end': 1166304}, {'start': 1167904, 'end': 1219040}, {'start': 1264672, 'end': 1287136}, {'start': 1301536, 'end': 1308128}]


In [ ]:
# Parameters
audio_sr = 16000  # Silero VAD sampling rate
video_fps = 25    # Replace with actual video FPS if different

# Convert VAD timestamps from audio samples → seconds → frame numbers
vad_frames = []
for ts in speech_timestamps:
    start_frame = int(ts['start'] / audio_sr * video_fps)
    end_frame = int(ts['end'] / audio_sr * video_fps)
    vad_frames.append((start_frame, end_frame))

print(vad_frames)


[(294, 335), (344, 427), (445, 475), (486, 584), (588, 618), (620, 693), (696, 782), (788, 825), (876, 1071), (1421, 1439), (1535, 1555), (1612, 1718), (1726, 1779), (1786, 1822), (1824, 1904), (1976, 2011), (2033, 2043)]
